In [138]:
import numpy as np
import pandas as pd
from math import sqrt
import random
from pprint import pprint
from collections import Counter 
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from binarytree import tree
from binarytree import Node
import sys

In [328]:
class dtree:
    train_np=np.empty(2)
    validate_np=np.empty(2)
    validate_label=[]
    cont_feature =['YrSold','MoSold','MiscVal','PoolArea','ScreenPorch','3SsnPorch','EnclosedPorch','OpenPorchSF','WoodDeckSF','GarageArea','GarageCars','GarageYrBlt','Fireplaces','TotRmsAbvGrd','Kitchen','Bedroom','LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath']
    cont_feature = set(cont_feature) 
    COLUMN_NAME=[]
    tree={}
    def clean_data(self,df) :
        df=df.drop(["Id", "Alley", "PoolQC", "Fence", "MiscFeature"],axis=1)
        new_LF=df.LotFrontage.mean()
        new_MV=df.MasVnrArea.mean()
        new_GY=df.GarageYrBlt.mean()
        new_MVT=df.MasVnrType.mode()[0]
        new_BQ=df.BsmtQual.mode()[0]
        new_BC=df.BsmtCond.mode()[0]
        new_BE=df.BsmtExposure.mode()[0]
        new_BF1=df.BsmtFinType1.mode()[0]
        new_BF2=df.BsmtFinType2.mode()[0]
        new_E=df.Electrical.mode()[0]
        new_FP=df.FireplaceQu.mode()[0]
        new_GT=df.GarageType.mode()[0]
        new_GF=df.GarageFinish.mode()[0]
        new_GQ=df.GarageQual.mode()[0]
        new_GC=df.GarageCond.mode()[0]

        replace_dict={"LotFrontage": new_LF,"MasVnrArea" : new_MV,"GarageYrBlt": new_GY,"BsmtQual" : new_BQ,"MasVnrType": new_MVT ,"BsmtCond": new_BC,"BsmtExposure": new_BE,"BsmtFinType1": new_BF1,
                      "BsmtFinType2": new_BF2,"Electrical": new_E,"FireplaceQu": new_FP,"GarageType": new_GT,"GarageFinish": new_GF,"GarageQual": new_GQ,"GarageCond": new_GC}
        df=df.fillna(replace_dict)
        return df
    
    def type_of_feature(self,feature):
        if feature in self.cont_feature:
            return "con"
        else:
            return "cat"
        
    
    def clean_data2(self,df):
        df=df.drop(["Id", "Alley", "PoolQC", "Fence", "MiscFeature"],axis=1)
        for col in df.columns:
            f_type=self.type_of_feature(col)
            if f_type=='con':
                new_val=df[col].mean()
                df=df.fillna({col : new_val})
            elif f_type=='cat':
                new_val=df[col].mode()[0]
                df=df.fillna({col : new_val})
        return df
    
    def split_data(self,train_csv):
        df=pd.read_csv(train_csv)
        df=self.clean_data2(df)
        Rows = df.shape[0] 
        test_size=100
        train_df=df.loc[:Rows-test_size-1]
        test_df=df.loc[Rows-test_size:]
        self.COLUMN_NAME=df.columns
        return train_df,test_df
  
    def create_leaf(self,data):
        prices= data[:,-1]
        
        leaf = np.mean(prices)
        return leaf
   
    def split_tree(self,train_np,split_column,split_value):
        column_values=train_np[:, split_column]
        f_type=self.type_of_feature(self.COLUMN_NAME[split_column])
        
        if f_type=='cat':
            data_left=train_np[column_values==split_value]
            data_right=train_np[column_values!=split_value]
        elif f_type=='con' :
            data_left=train_np[column_values<=split_value]
            data_right=train_np[column_values>split_value]
        return data_left,data_right
    
    def calc_mse(self,data_array):
        values=data_array[:,-1]
        if len(values)==0:
            return 0
        else :
            pred=np.mean(values)
            ms=np.mean((values - pred) **2)
        return ms
    
    def overall_mse(self,data_left,data_right):
        dl=len(data_left)
        dr=len(data_right)
        total=dl+dr
        pl=dl/total
        pr=dr/total
        
        vl=pl*self.calc_mse(data_left)
        vr=pr*self.calc_mse(data_right)
        
        return vl+vr
        
    def find_best_split(self,train_np):
        
        split_dict={}
        for feature in range(train_np.shape[1]-1):
            value=train_np[:,feature]
            split_array=np.unique(value)
            split_dict[feature]=split_array
#         print(len(split_dict))
        
        first_it=True
        for feature in split_dict:
            
            f_type=self.type_of_feature(self.COLUMN_NAME[feature])
#             print(self.COLUMN_NAME[feature]," ",f_type)
            if(f_type=='cat') :
                for val in split_dict[feature]:
                    data_left,data_right=self.split_tree(train_np,feature,val)
                    calc_mse=self.overall_mse(data_left,data_right)
                    if first_it==True:
                        best_score=calc_mse
                        split_column=feature
                        split_value=val
                        first_it=False
                    
                    if calc_mse<=best_score and first_it==False:
                        best_score=calc_mse
                        split_column=feature
                        split_value=val
            
            elif f_type=='con':
                for i in range (1,len(split_dict[feature])):
                    val=split_dict[feature][i]+split_dict[feature][i-1]
                    val=val/2
                    data_left,data_right=self.split_tree(train_np,feature,val)
                    calc_mse=self.overall_mse(data_left,data_right)
                    if first_it==True:
                        best_score=calc_mse
                        split_column=feature
                        split_value=val
                        first_it=False
                    
                    if calc_mse<=best_score and first_it==False:
                        best_score=calc_mse
                        split_column=feature
                        split_value=val
#             print(best_score)
        return split_column,split_value
        
    def DTAlgo(self,train_np,count,Max_Depth=5):
        
        if(count==Max_Depth) or train_np.shape[0]<=2:
                return self.create_leaf(train_np)
        else:
            count=count+1
            split_column,split_val=self.find_best_split(train_np)
#             print(split_column," ",split_val)
            split_left,split_right=self.split_tree(train_np,split_column,split_val)
            
            query="{} {}".format(split_column,split_val)
            sub_tree={query:[]}
            left=self.DTAlgo(split_left,count,Max_Depth)
            right=self.DTAlgo(split_right,count,Max_Depth)
            sub_tree[query].append(left)
            sub_tree[query].append(right)

            return sub_tree
    
    def validate_data(self):
        predicted=[]
        self.validate_label=self.validate_np[:,-1]
        col=self.validate_np.shape[1]
        validate_np1=self.validate_np[:,:col-1]
#         print(validate_np1.shape)
        for test in validate_np1 :
            result=self.iterate_tree(test,self.tree)
            predicted.append(result)
        return predicted
        

    
    def train(self,train_csv):
        train_df,validate_df=self.split_data(train_csv)
         
        self.train_np=train_df.to_numpy()
#         print(self.train_np.shape)
        self.validate_np=validate_df.to_numpy()
        self.tree=self.DTAlgo(self.train_np,0)
#         print(self.tree)
        validate_result=self.validate_data()
        print ("validation MSE : ",mean_squared_error(self.validate_label, validate_result))
        
    
    def iterate_tree(self,test,tree):
        if not isinstance(tree,dict):
            return tree
        
        query1 = list(tree.keys())[0] 
        query =query1.split(' ')
        feature=self.COLUMN_NAME[int(query[0])]
        
        f_type=self.type_of_feature(feature)
        tree_check=query[1]
        test_check=test[int(query[0])]

        if(f_type=='con'):
            if(test_check <= float(tree_check)):
                return self.iterate_tree(test,tree[query1][0])
            else:
                return self.iterate_tree(test,tree[query1][1])
        
        elif f_type=='cat':
            if(test_check==tree_check):
                return self.iterate_tree(test,tree[query1][0])
            else:
                return self.iterate_tree(test,tree[query1][1])
        
            

    def predict(self,test_csv):
        predicted=[]
        test_df=pd.read_csv(test_csv)
        test_df=self.clean_data2(test_df)
        test_np=test_df.to_numpy()
        for test in test_np:
            result=self.iterate_tree(test,self.tree)
            predicted.append(result)
        return predicted
        

        

In [329]:
dtree_regressor = dtree()
dtree_regressor.train('./Datasets/q3/train.csv')
predictions = dtree_regressor.predict('./Datasets/q3/test.csv')
test_labels = list()
with open("./Datasets/q3/test_labels.csv") as f:
  for line in f:
    test_labels.append(float(line.split(',')[1]))
print(predictions[:6])
print(test_labels[:6])
print (mean_squared_error(test_labels, predictions))

validation MSE :  2796723815.589552
[134428.90217391305, 115385.02205882352, 226050.0, 148503.38356164383, 132275.16666666666, 134428.90217391305]
[128000.0, 124900.0, 191000.0, 144000.0, 231500.0, 135750.0]
2099837083.3796349
